TSM - Assignment.

Collect the data for real and nominal yields from the Federal Reserve covering the period Jan 2005-August 2022.

In [ ]:
import pandas as pd
from helpers import plotYield
import plotly.graph_objects as go
import plotly.express as px
from statsmodels.tsa.stattools import adfuller
import chart_studio.plotly as py
import plotly.offline as pyo
import plotly.io as pio
import numpy as np

from datamanipulation import dataManipulation
from statTests import statTest
from yieldMaturitySelector import yieldMatSelector

# trick
pio.renderers.default = "notebook"

# instantiate classes 
yieldselector = yieldMatSelector()
stattest = statTest()
data_manip = dataManipulation()

# collect data from files
nominal_yields = pd.read_excel('nominal_yields.xlsx').dropna()
real_yields = pd.read_excel('real_yields.xlsx').dropna()



In [ ]:
# show the first 5 rows of nominal yields
nominal_yields.head()

In [ ]:
# show the first 5 rows of real yields
real_yields.head()

In [ ]:
# select specific maturities
nominal_yields_2_10y_eom = yieldselector.adjustYieldSerie(df=nominal_yields,type='nominal',FD=False)
real_yields_2_10y_eom = yieldselector.adjustYieldSerie(df=real_yields,type='real',FD=False)

In [ ]:
# plot nominal data
plotYield(nominal_yields_2_10y_eom,columns=['SVENY02','SVENY03','SVENY05','SVENY07','SVENY10'],type='Nominal',FD=False)

In [ ]:
# plot real yield data (TIPS)
plotYield(real_yields_2_10y_eom,columns=['TIPSY02','TIPSY03','TIPSY05','TIPSY07','TIPSY10'],type='Real',FD=False)

We have extracted and plotted the data, but before proceeding with PCA, it is usually well-practice to ensure stationarity of the data under consideration. From the figures of nominal and real yields respectively, some clear trends which for the statistical inference of data analysis should be removed (or limited as much as possible) for a correct interpretation of the results.

In [ ]:
# check for stationarity in nominal yields
nominal2y_yieldADF = stattest.ADFtest(nominal_yields_2_10y_eom,type='nominal',maturity='2y')

In [ ]:
# let's try for 5y yields
nominal5y_yieldADF = stattest.ADFtest(nominal_yields_2_10y_eom,type='nominal',maturity='5y')

In [ ]:
# let's try for 10y yields
nominal10y_yieldADF = stattest.ADFtest(nominal_yields_2_10y_eom,type='nominal',maturity='10y')

In [ ]:
# nominal yields are non-stationary, so we re-take the original data, apply first difference and then take only the month end
nominal_yields_2_10y_eom_FD = yieldselector.adjustYieldSerie(nominal_yields,type='nominal',FD=True)
nominal_yields_2_10y_eom_FD.head()

In [ ]:
# check stationarity after first difference (only for the 2y)
nominal2y_yieldADF_FD = stattest.ADFtest(nominal_yields_2_10y_eom_FD,type='nominal',maturity='2y')
#ADFtest(nominal_yields_2_10y_eom_shifted,type='nominal',maturity='2y')

In [ ]:
# let's plot the nominal yields stationary data
plotYield(nominal_yields_2_10y_eom_FD,columns=['SVENY02','SVENY03','SVENY05','SVENY07','SVENY10'],type='Nominal',FD=True)

In [ ]:
# check for stationarity in real yields
real2y_yieldADF_FD = stattest.ADFtest(real_yields_2_10y_eom,type='real',maturity='2y')

In [ ]:
# let's try with 5y
real5y_yieldADF_FD = stattest.ADFtest(real_yields_2_10y_eom,type='real',maturity='5y')

In [ ]:
# let's try with 10y
real10y_yieldADF_FD = stattest.ADFtest(real_yields_2_10y_eom,type='real',maturity='10y')

In [ ]:
# real yields are non-stationary, so we re-take the original data, apply first difference and then take only the month end
real_yields_2_10y_eom_FD = yieldselector.adjustYieldSerie(real_yields,type='real',FD=True)

In [ ]:
# check stationarity after first difference (only for the 2y)
real2y_yieldADF_FD = stattest.ADFtest(real_yields_2_10y_eom_FD,type='real',maturity='2y')

In [ ]:
# let's plot the nominal yields stationary data
plotYield(real_yields_2_10y_eom_FD,columns=['TIPSY02','TIPSY03','TIPSY05','TIPSY07','TIPSY10'],type='Real',FD=True)

Now we have done all the preliminary analysis entailing first differencing the data to ensure stationarity. At this point, we can start with the principal components analysis (PCA) using the stationary data.

In [ ]:
# PCA